In [1]:
import mne
from mne.preprocessing import ICA
# raw_bv = mne.io.read_raw_brainvision('rest\INP0064_v1.4_rs11_S008_R003_04.12.23.vhdr')
raw_bv = mne.io.read_raw_brainvision('rest1_94/INP0094_v1.4_REST1_Op008_A002_S002_18.12.23.vhdr')
raw_bv.load_data()

In [2]:
# print(mne.get_config())

In [3]:
# print(raw_bv.info)
# print(raw_bv.ch_names)

In [4]:
# raw_bv.describe()

In [5]:
raw_bv_dropped = raw_bv.copy()

raw_bv_dropped.drop_channels(["EOG", "BIP1", "M1", "M2", "PO5", "PO6"])
# raw_bv_dropped.drop_channels(["BIP1"])

# raw_bv_dropped.rename_channels()

In [6]:
# %matplotlib qt

# raw_bv_dropped.plot()

In [7]:
# raw_bv.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)
# raw_bv.plot(duration=5, n_channels=30)
raw_bd_filtered = raw_bv_dropped.copy()
raw_bd_filtered = raw_bd_filtered.filter(l_freq=1, h_freq=40)

In [8]:
%matplotlib qt
# fig = raw.plot(start=2, duration=6)

In [9]:
# raw_bv.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)

In [10]:
biosemi64 = mne.channels.make_standard_montage("biosemi64")
# customMontage  = mne.channels.read_custom_montage("rest1_94\\reworkedElc.elc")

In [11]:
# biosemi64.plot()  #

# fig = biosemi64.plot(kind="3d", show=False)  # 3D
# fig = fig.gca().view_init(azim=70, elev=15)  # set view angle for tutorial
# customMontage.plot()

In [12]:
raw_bd_filtered.set_montage(biosemi64)

In [13]:
raw_bdf_reref_obj = mne.set_eeg_reference(raw_bd_filtered)
raw_bdf_reref = raw_bdf_reref_obj[0]
# raw_bdf_reref.plot(title="reref")
# raw_bv_dropped.plot(title="original")

In [14]:
ica = ICA(n_components=20, random_state=42, max_iter='auto')
ica.fit(raw_bdf_reref)


In [15]:
# expl_var = ica.get_explained_variance_ratio(raw_reref)
# for channel_type, ratio in expl_var.items():
#     print(
#         f"Fraction of {channel_type} variance explained by all components: " f"{ratio}"
#     )

In [16]:
# ica.plot_sources(raw_reref, start=2)
# raw_reref.plot(start=0, duration=20)

In [17]:
#ica.plot_components()

In [18]:
# ica.plot_properties(raw_reref, picks=range(0, 9))

In [19]:
# ica.exclude = [0, 1, 7, 14, 16, 17] # 30 components
# ica.exclude = [0, 3, 5, 14, 13, 56] 
ica.exclude = [0, 1, 4, 7, 11, 13, 15, 19]
raw_bdfr_reconst = raw_bdf_reref.copy()
ica.apply(raw_bdfr_reconst)

In [20]:
# raw_bdfr_reconst.plot(title="ica")
# raw_bv_dropped.plot(title="original")
# raw2 = raw_bv.copy()
# raw2.filter(l_freq=1, h_freq=30)
# raw2.plot(title="1-30")

In [21]:
# %matplotlib inline
raw_bv_reconst_reref = raw_bdfr_reconst.copy()
raw_bv_reconst_reref.set_montage(biosemi64)
# for i in range(0, 50):
#     tmin=(0.1*i)
#     tmax=(1+(0.1*i))
#     print(f'tmin = {tmin}; tmax = {tmax}')
#     spectrum = raw_bv_reconst_reref.compute_psd(fmin=1, fmax=40, tmin=tmin, tmax=tmax)
#     spectrum.plot_topomap(bands={'Theta (4-8 Hz)': (4, 8)})
#     spectrum.plot_topomap(bands={'Delta (0-4 Hz)': (0, 4), 'Theta (4-8 Hz)': (4, 8),
#          'Alpha (8-12 Hz)': (8, 12), 'Beta (12-30 Hz)': (12, 30),
#          'Gamma (30-45 Hz)': (30, 45)})
    # spectrum.plot()

    #построить поочередные картинки


In [22]:
def raw_compute_psd(raw, tmin, tmax):
    return raw.compute_psd(
        fmin=1,
        fmax=40,
        tmin=tmin,
        tmax=tmax,
        method='welch',
        n_fft=2048,
        window='hann',
        n_per_seg=2048,
        n_overlap = 0,
    )

In [23]:
import pandas as pd

def calc_one_second_spectrum(raw, time_start, time_end):
    spectrum = raw_compute_psd(
        raw=raw,
        tmin=time_start,
        tmax=time_start + 1
    )
    aggregate_df = spectrum.to_data_frame()
    aggregate_df["tmin"] = time_start
    aggregate_df["tmax"] = time_start + 1
    for i in range(time_start+1, (time_end*2)-1):
        tmin = i/2
        tmax = tmin+1
        print(f"tmin: {tmin} | tmax: {tmax}")
        spectrum = raw_compute_psd(
            raw=raw_bv_reconst_reref,
            tmin=tmin,
            tmax=tmax
        )
        df = spectrum.to_data_frame()
        df["tmin"] = tmin
        df["tmax"] = tmax
        aggregate_df = pd.concat([aggregate_df, df], ignore_index=True, axis=0)
    return aggregate_df


In [24]:
aggregate_df = calc_one_second_spectrum(
    raw = raw_bv_reconst_reref,
    time_start=0,
    time_end=30
)

In [25]:
# jupyter-pavel@192.168.88.244
# jira, nextcloud, confluence
# d 1-4
# t 4-8 4-6 6-8
# a 8-13 8-11 11-13
# b 13-30 13-20 20-25 25-30
# g 30-45
# TODO: проверить пересечения сегментов!!!
freq_dict = [
    ('delta', [1, 4]),

    ('theta-a', [4, 8]),
    ('theta-1', [4, 6]),
    ('theta-2', [6, 8]),

    ('alpha-a', [8, 13]),
    ('alpha-1', [8, 11]),
    ('alpha-2', [11, 13]),

    ('beta-a', [13, 30]),
    ('beta-1', [13, 20]),
    ('beta-2', [20, 25]),
    ('beta-3', [25, 30]),

    ('gamma', [30, 45])
]

# aggregate_df

In [26]:
def calc_mean_one_band(spectrum_df, freq_name, bandwidth_start, bandwidth_end, tmin):
    mean_df = spectrum_df \
        .loc[spectrum_df['freq'].isin(range(bandwidth_start, bandwidth_end + 1))] \
        .loc[spectrum_df['tmin'] == tmin] \
        .mean(axis=0)\
        .to_frame().T

    mean_df = mean_df.drop('freq', axis=1)

    mean_df['bandwidth_start'] = bandwidth_start
    mean_df['bandwidth_end'] = bandwidth_end
    mean_df['bandwidth_name'] = freq_name
    return mean_df

In [27]:
def calc_mean_for_bands(spectrum_df, freq_dict, time_start, time_end):
    mean_agr_df = pd.DataFrame()
    for freq_tup in freq_dict:
        freq_name = freq_tup[0]
        freqs = freq_tup[1]
        bandwidth_start = freqs[0]
        bandwidth_end = freqs[1]
        for i in range(time_start, (time_end*2)-1):
            tmin = i/2
            df = calc_mean_one_band(
                spectrum_df=spectrum_df,
                freq_name=freq_name,
                bandwidth_start=bandwidth_start,
                bandwidth_end=bandwidth_end,
                tmin=tmin
            )
            mean_agr_df = pd.concat([mean_agr_df, df], ignore_index=True, axis=0)
    return mean_agr_df

In [28]:
def calc_one_second_spectrum_and_freq_mean(raw, freq_dict, time_start, time_end):
    spectrum_df = calc_one_second_spectrum(
        raw=raw,
        time_start=time_start,
        time_end=time_end
    )
    return calc_mean_for_bands(
        spectrum_df=spectrum_df,
        freq_dict=freq_dict,
        time_start=time_start,
        time_end=time_end
    )

In [29]:
mean_agr_df = calc_one_second_spectrum_and_freq_mean(raw_bv_reconst_reref, freq_dict, 0, 60)

mean_agr_df.set_index(['tmin', 'bandwidth_name'])

In [30]:
mne.events_from_annotations(raw_bv_reconst_reref)[1]

In [31]:
mne.events_from_annotations(raw_bv_reconst_reref)[0]

# Calculated EDF

In [32]:
edf = mne.io.read_raw_edf('./rest1_94/Test_01.EDF')

In [33]:
spectrum_edf = edf.compute_psd(
    fmin=1, 
    fmax=40, 
    tmin=0, 
    tmax=11, 
    method='welch',
    n_fft=2048,
    window='hann',
    n_per_seg=2048,
    n_overlap = 1024,
)

In [34]:
df = spectrum_edf.to_data_frame()
mean = df.mean(axis=0)
df2 = pd.concat([df.loc[:], mean.to_frame().T], ignore_index=True)
# df.append()
# df2
mean_frame = mean.to_frame().T
mean_frame_val = mean_frame.loc[:, mean_frame.columns!='freq']
mean_frame_val

# mean_frame_val.plot()

In [35]:
df

In [36]:
import math
import numpy as np

%matplotlib inline

#INFO: Проверка того как данные вообще хранятся

df_map = df.map(lambda x: (pow(10, x))-1)

# df_map = df.map(lambda x: np.float_power(np.float64(10), x))
df_val = df_map.loc[4:, df.columns!='freq']
# df_val.plot(legend=False, figsize=(12,4), linewidth=0.5)
spectrum_edf.plot(dB=False, amplitude=False)
spectrum_edf.plot(dB=True, amplitude=False)
# df_val2 = df.loc[:, df.columns!='freq']
# df_map2 = df.apply(lambda x: x.map(lambda y: (y if y >= 1 else math.sqrt(y * x['freq']))), axis=1)
# df_map3 = df_map2.apply(lambda x: x.map(lambda y: (y if y >= 1 else (y / x['freq']))), axis=1)
# df_val2 = df_map2.loc[3:, df.columns!='freq']
# df_val3 = df_map3.loc[1:, df.columns!='freq']


# df_val2.plot(legend=False, figsize=(12,4), linewidth=0.5)
df_val4 = df.loc[3:, df.columns!='freq']
df_val4.plot(legend=False, figsize=(12,4), linewidth=0.5)